## Problem Statement


##### We are suppossed to build a model that has the capacity to generate words on their own, when we send a set of input words to and we 
##### need to classify them as UFDP(Unfair and Deceptive business practice related) or not UFDP.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [2]:
train= pd.read_csv("GrievancesData_Train.csv") #loading train data
train.head()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,...",Mortgage,Closed with explanation,No,2016-01-30
1,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XX...,Mortgage,Closed with explanation,Yes,2016-02-10
2,GID512413,Bank5286,State14,2016-01-19,Billing disputes,"In XXXX XXXX, I was given a {$1100.00} credit...",Credit card,Closed with monetary relief,Yes,2016-02-10
3,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage was sold to Bank5279 in XXXX of ...,Mortgage,Closed with explanation,No,2016-01-30
4,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in...,Mortgage,Closed with explanation,No,2016-01-30


In [3]:
train.describe()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
count,53680,53680,53559,53680,53680,53680,53680,53680,53680,53680
unique,53680,86,59,1222,67,46662,4,4,2,1248
top,GID364172,Bank5286,State9,2014-01-19,"Loan servicing, payments, escrow account",I was shocked when I reviewed my credit repor...,Credit card,Closed with explanation,No,2013-10-01
freq,1,5623,8110,172,8455,8,21638,41546,41895,142


In [98]:
test= pd.read_csv("GrievancesData_Test.csv")
test.head()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30
1,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03
2,GID515121,Bank5372,State56,2016-03-19,"Loan servicing, payments, escrow account",We fell behind in our payments back in 2014 f...,Mortgage,Closed with explanation,No,2016-03-30
3,GID515123,Bank5372,State47,2016-03-19,"Loan servicing, payments, escrow account",My home is on XXXX parcels of land. The first...,Mortgage,Closed with explanation,No,2016-03-30
4,GID515124,Bank5310,State10,2016-03-19,Settlement process and costs,We mortgaged our home with XXXX on XXXX/XXXX/...,Mortgage,Closed with explanation,No,2016-03-30


In [5]:
train.shape,test.shape

((53680, 10), (27954, 10))

In [99]:
test.describe()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
count,27954,27954,27879,27954,27954,27954,27954,27954,27954,27954
unique,27954,22,61,1168,59,24137,4,4,2,1198
top,GID444548,Bank5373,State9,2014-01-19,"Loan servicing, payments, escrow account",I was shocked when I reviewed my credit repor...,Credit card,Closed with explanation,No,2014-02-19
freq,1,8221,3839,727,4403,6,9561,23505,22524,291


In [7]:
## Calculating No of Words for train

train["word_count"]=train["GrievanceDescription"].apply(lambda x: len(str(x).split(" ")))
train[["word_count","GrievanceDescription"]].head()

,word_count,GrievanceDescription
0,169,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,..."
1,515,I had a mortgage with from XXXX of XXXX to XX...
2,569,"In XXXX XXXX, I was given a {$1100.00} credit..."
3,284,Our Mortgage was sold to Bank5279 in XXXX of ...
4,422,We started dealing with Citi Mortgage back in...


In [8]:
## Calculating No of Characters in each line

train["char_count"]=train["GrievanceDescription"].str.len()  # also includes spaces
train[["char_count","GrievanceDescription"]].head()

,char_count,GrievanceDescription
0,944,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,..."
1,2641,I had a mortgage with from XXXX of XXXX to XX...
2,2873,"In XXXX XXXX, I was given a {$1100.00} credit..."
3,1456,Our Mortgage was sold to Bank5279 in XXXX of ...
4,2281,We started dealing with Citi Mortgage back in...


In [9]:
##to check the number of stopwords in each Description

from nltk.corpus import stopwords
stop = stopwords.words('english')

train['GrievanceDescription'] = train['GrievanceDescription'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['GrievanceDescription'].head()


0    Bank5279 Research dpt. XXXX XXXX XXXX XXXX, CA...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX, I given {$1100.00} credit limit ...
3    Our Mortgage sold Bank5279 XXXX 2014. They bou...
4    We started dealing Citi Mortgage back XX/XX/XX...
Name: GrievanceDescription, dtype: object

In [10]:
##Number of Special Characters for train

train['hastags'] = train['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['GrievanceDescription','hastags']].head()


,GrievanceDescription,hastags
0,"Bank5279 Research dpt. XXXX XXXX XXXX XXXX, CA...",0
1,I mortgage XXXX XXXX XXXX XXXX never missed pa...,0
2,"In XXXX XXXX, I given {$1100.00} credit limit ...",0
3,Our Mortgage sold Bank5279 XXXX 2014. They bou...,0
4,We started dealing Citi Mortgage back XX/XX/XX...,0


In [166]:
##Number of Special Characters for test

test['hastags'] = test['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
test[['GrievanceDescription','hastags']].head()

,GrievanceDescription,hastags
0,I currently have a mortgage with Flag star ba...,0
1,Bank5278 Auto Finance repossessed and subsequ...,0
2,We fell behind in our payments back in 2014 f...,0
3,My home is on XXXX parcels of land The first ...,0
4,We mortgaged our home with XXXX on XXXXXXXXXX...,0


In [11]:
## Removing Punctuation

train['GrievanceDescription'] = train['GrievanceDescription'].str.replace('[^\w\s]','')
train["GrievanceDescription"].head()

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX I given 110000 credit limit Sears...
3    Our Mortgage sold Bank5279 XXXX 2014 They boug...
4    We started dealing Citi Mortgage back XXXXXXXX...
Name: GrievanceDescription, dtype: object

In [12]:
## No of UpperCase words

train['upper'] = train['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

train[['GrievanceDescription','upper']].head(15)


,GrievanceDescription,upper
0,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,31
1,I mortgage XXXX XXXX XXXX XXXX never missed pa...,93
2,In XXXX XXXX I given 110000 credit limit Sears...,53
3,Our Mortgage sold Bank5279 XXXX 2014 They boug...,29
4,We started dealing Citi Mortgage back XXXXXXXX...,19
5,Trouble payments handled My husband lost job X...,57
6,TOO BIG TO FAIL AND CARE We XXXX last child we...,43
7,I car loan Bank5397 In XXXX 2014 I sent paymen...,7
8,Ocwen loan servicing servicer loan 1 They refu...,22
9,We trying close new Mortgage applied for We no...,53


In [13]:
## Rare words removal for train

freq = pd.Series(' '.join(train['GrievanceDescription']).split()).value_counts()[-10:]
freq

Roundpint         1
beeb              1
ANOTER            1
2We               1
deednote          1
ENACTED           1
PAgain            1
AirBag            1
tohas             1
Securitisation    1
dtype: int64

In [14]:
##The frequency of the rare words for train
freq = list(freq.index)
train['GrievanceDescription'] = train['GrievanceDescription'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['GrievanceDescription'].head()

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX I given 110000 credit limit Sears...
3    Our Mortgage sold Bank5279 XXXX 2014 They boug...
4    We started dealing Citi Mortgage back XXXXXXXX...
Name: GrievanceDescription, dtype: object

In [15]:
## Removing Special Characters

train['hastags'] = train['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['GrievanceDescription','hastags']].head()

,GrievanceDescription,hastags
0,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,0
1,I mortgage XXXX XXXX XXXX XXXX never missed pa...,0
2,In XXXX XXXX I given 110000 credit limit Sears...,0
3,Our Mortgage sold Bank5279 XXXX 2014 They boug...,0
4,We started dealing Citi Mortgage back XXXXXXXX...,0


In [16]:
from textblob import TextBlob

In [17]:
##TextBlob is used for Spelling Correction

train["GrievanceDescription"][:5].apply(lambda x: str(TextBlob(x).correct()))
train["GrievanceDescription"].head()

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX I given 110000 credit limit Sears...
3    Our Mortgage sold Bank5279 XXXX 2014 They boug...
4    We started dealing Citi Mortgage back XXXXXXXX...
Name: GrievanceDescription, dtype: object

In [18]:
train["GrievanceDescription"].head(10)

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX I given 110000 credit limit Sears...
3    Our Mortgage sold Bank5279 XXXX 2014 They boug...
4    We started dealing Citi Mortgage back XXXXXXXX...
5    Trouble payments handled My husband lost job X...
6    TOO BIG TO FAIL AND CARE We XXXX last child we...
7    I car loan Bank5397 In XXXX 2014 I sent paymen...
8    Ocwen loan servicing servicer loan 1 They refu...
9    We trying close new Mortgage applied for We no...
Name: GrievanceDescription, dtype: object

### Text Preprocessing 

In [19]:
##Tokenizaion

TextBlob(train["GrievanceDescription"][1]).words

WordList(['I', 'mortgage', 'XXXX', 'XXXX', 'XXXX', 'XXXX', 'never', 'missed', 'payment', 'I', 'refinanced', 'Citizens', 'Mortgage', 'XXXX', 'XXXX', 'XXXX', 'obtain', 'XXXX', 'loan', 'completely', 'renovate', 'home', 'My', 'home', 'built', 'My', 'current', 'mortgage', 'XXXX', '41000000', 'My', 'house', 'single', 'family', 'XXXX', 'XXXX', 'XXXX', 'ocean', 'view', 'XXXX', 'car', 'garage', 'additional', 'parking', 'spot', 'In', 'late', 'XXXX', 'XXXX', 'I', 'moved', 'in', 'I', 'started', 'refinancing', 'process', 'Citizens', 'lower', 'monthly', 'payment', 'close', '60000', 'I', 'would', 'longer', 'need', 'PMI', 'insurance', 'I', 'provided', 'Citizens', 'every', 'document', 'asked', 'ASAP', 'XXXX', 'XXXX', 'XXXX', 'XXXX', 'I', 'signed', 'allow', 'income', 'taxes', 'released', 'I', 'paid', '45000', 'appraisial', 'home', 'occurred', 'XXXX', 'XXXX', 'XXXX', 'The', 'apprasal', 'came', '59000000', 'signifigantly', 'market', 'value', 'I', 'raised', 'complaints', 'appraisal', 'loan', 'originator', 

In [20]:
## Lemmatization( The process of converting a word to its root form from the Whole Dataset)

from textblob import Word
train["GrievanceDescription"]=train["GrievanceDescription"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train["GrievanceDescription"].head()

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX I given 110000 credit limit Sears...
3    Our Mortgage sold Bank5279 XXXX 2014 They boug...
4    We started dealing Citi Mortgage back XXXXXXXX...
Name: GrievanceDescription, dtype: object

In [21]:
train.Grievance_Category.value_counts() # checks the count of values in the Column

Loan servicing, payments, escrow account                                            8455
Loan modification,collection,foreclosure                                            6469
Billing disputes                                                                    3645
Dealing with my lender or servicer                                                  3171
Managing the loan or lease                                                          2885
Other                                                                               2142
Identity theft / Fraud / Embezzlement                                               1951
Closing/Cancelling account                                                          1599
Application, originator, mortgage broker                                            1557
Trouble during payment process                                                      1342
Problems when you are unable to pay                                                 1170
Customer service / Cu

In [22]:
train.nunique() # checking the unique values in the train data

GrievanceID             53680
BankID                     86
State                      59
DateOfGrievance          1222
Grievance_Category         67
GrievanceDescription    43887
LineOfBusiness              4
ResolutionComments          4
Disputed                    2
DateOfResolution         1248
word_count               1008
char_count               4142
hastags                     1
upper                     347
dtype: int64

In [23]:
train.GrievanceDescription.value_counts() # For Train Data Column

I shocked I reviewed credit report found late payment date 30 day past due XXXX 2016 I sure happened I believe I made payment I received statement My thought monthly statement get me                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [24]:
##Finding the total number of unique values in each column

train.apply(lambda x: len(x.unique()))

GrievanceID             53680
BankID                     86
State                      60
DateOfGrievance          1222
Grievance_Category         67
GrievanceDescription    43887
LineOfBusiness              4
ResolutionComments          4
Disputed                    2
DateOfResolution         1248
word_count               1008
char_count               4142
hastags                     1
upper                     347
dtype: int64

In [25]:
train.columns.unique()

Index(['GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'GrievanceDescription', 'LineOfBusiness',
       'ResolutionComments', 'Disputed', 'DateOfResolution', 'word_count',
       'char_count', 'hastags', 'upper'],
      dtype='object')

In [28]:
tar= pd.read_csv("Train.csv")

In [29]:
tar.head()

,BankID,BankGrade
0,Bank5298,satisfactory
1,Bank5421,deficient
2,Bank5326,satisfactory
3,Bank5432,outstanding
4,Bank5439,deficient


In [30]:
##Advanced Text Processing

## Generating N Grams

TextBlob(train['GrievanceDescription'][0]).ngrams(2)



[WordList(['Bank5279', 'Research']),
 WordList(['Research', 'dpt']),
 WordList(['dpt', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'CA']),
 WordList(['CA', 'XXXXXXXX']),
 WordList(['XXXXXXXX', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'name']),
 WordList(['name', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'I']),
 WordList(['I', 'customer']),
 WordList(['customer', 'loan']),
 WordList(['loan', 'I']),
 WordList(['I', 'refinanced']),
 WordList(['refinanced', 'home']),
 WordList(['home', 'XXXX']),
 WordList(['XXXX', 'The']),
 WordList(['The', 'new']),
 WordList(['new', 'loan']),
 WordList(['loan', 'funded']),
 WordList(['funded', 'XXXXXXXX15']),
 WordList(['XXXXXXXX15', 'I']),
 WordList(['I', 'included']),
 WordList(['included', 'copy']),
 WordList(['copy', 'XXXXXXXX15']),
 WordList(['XXXXXXXX15', 'statement']),
 WordList(['statement', 'Bank5279']),
 WordList(['Bank5279', 'HUD']),
 WordLi

In [31]:
## Constructing TFIDF Matrix
from sklearn.feature_extraction.text import TfidfVectorizer
train.columns

Index(['GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'GrievanceDescription', 'LineOfBusiness',
       'ResolutionComments', 'Disputed', 'DateOfResolution', 'word_count',
       'char_count', 'hastags', 'upper'],
      dtype='object')

In [32]:
list_Gr= []  # Storing the Column vectors  in a List for Train 
for i in range(len(train["GrievanceDescription"])):
    list_Gr.append(train["GrievanceDescription"][i])

In [33]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

X_train_tfidf = tfidf_transformer.fit_transform(list_Gr)

X_test_tfidf = tfidf_transformer.fit_transform(list_Gr)

In [102]:
list_Gr1= []  # Storing the Column vectors  in a List for Test
for i in range(len(test['GrievanceDescription'])):
    list_Gr1.append(test['GrievanceDescription'][i])

In [103]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1), stop_words='english') # Using the TFIDF Vectorizer

X_train_tfidf = tfidf_transformer.fit_transform(list_Gr1)

X_test_tfidf = tfidf_transformer.fit_transform(list_Gr1)

In [107]:
X_test_tfidf

<27954x29521 sparse matrix of type '<class 'numpy.float64'>'
	with 1748137 stored elements in Compressed Sparse Row format>

In [108]:
type(X_train_tfidf)

scipy.sparse.csr.csr_matrix

In [109]:
type(X_test_tfidf)

scipy.sparse.csr.csr_matrix

In [110]:
X_train_tfidf.shape


(27954, 29521)

In [111]:
X_test_tfidf.shape

(27954, 29521)

In [40]:
len(tfidf_transformer.get_feature_names())


53290

In [41]:
tfidf_transformer.get_feature_names()


['00',
 '000',
 '0000',
 '00000',
 '000000',
 '0001',
 '000balance',
 '000interest',
 '000xxxxxxxx',
 '001',
 '001informed',
 '005',
 '01',
 '010',
 '01000',
 '011',
 '012',
 '0120',
 '0125',
 '014',
 '019',
 '025',
 '03',
 '035',
 '0395',
 '05',
 '062016',
 '0625',
 '064',
 '068',
 '07',
 '078',
 '08',
 '0875',
 '09',
 '0901',
 '0945',
 '0cwen',
 '0date',
 '0f',
 '0k',
 '0monthly',
 '0n',
 '0nxxxxxxxx',
 '0r',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000000',
 '10000000',
 '100000000',
 '1000000000',
 '1000month',
 '1001',
 '1002',
 '1002240',
 '10029',
 '10029notifications',
 '1003',
 '10032',
 '10034',
 '10035',
 '10038',
 '1005',
 '100510',
 '1007',
 '1009',
 '100k',
 '100mo',
 '100month',
 '100th',
 '100xxxx',
 '101',
 '1011',
 '1012',
 '10125',
 '1012business',
 '10136',
 '1014',
 '1015',
 '1015days',
 '1017',
 '102',
 '1020',
 '102242',
 '1024',
 '102417',
 '102433',
 '102434',
 '102435',
 '102435102436',
 '102435mortgage',
 '102436',
 '102437',
 '102438',
 '102439',
 '10

In [112]:
from sklearn.cluster import KMeans  # importing library

In [113]:
Cluster1 = KMeans(n_clusters=2) # Cluster 1 for the Train data
Cluster2= KMeans(n_clusters=2) # Cluster 2 for Test Data

In [44]:
predicttrain= Cluster1.fit(X_train_tfidf) # Fitting the train cluster to the X_train_tfidf matrix

In [45]:
predicttest= Cluster2.fit(X_test_tfidf) # Fitting the test cluster to the X_train_tfidf matrix

In [ ]:
predicttrain1= Cluster1.fit(X_train_tfidf) # For Test Data fitting the Test tfidf matrix
predicttest1= Cluster2.fit(X_tfidf_test)  # For Test Data fitting the Test tfidf matrix

In [46]:
cluster_test = predicttrain.labels_.tolist() # Fitting the Cluster Data to the list Created

In [117]:
cluster_test1= predicttrain.labels_.tolist() # Fitting the Cluster Data to the list Created for Test Data


In [168]:
cluster_tr1 = predicttrain.labels_.tolist()  # Fitting the Cluster Data to the list Created for Train Data

In [118]:
cluster_test1 # Checking the test Cluster

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [48]:
cluster_tr = predicttrain.labels_.tolist() # Fitting the Cluster Data to the list Created

In [119]:
cluster_tr1  # Checking the test Cluster

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [50]:
test= pd.read_csv("GrievancesData_Test.csv")

In [146]:
## Calculating No of Words for test

test["word_count"]=test["GrievanceDescription"].apply(lambda x: len(str(x).split(" ")))
test[["word_count","GrievanceDescription"]].head(15)

,word_count,GrievanceDescription
0,110,I currently have a mortgage with Flag star ba...
1,79,Bank5278 Auto Finance repossessed and subsequ...
2,374,We fell behind in our payments back in 2014 f...
3,436,My home is on XXXX parcels of land. The first...
4,90,We mortgaged our home with XXXX on XXXX/XXXX/...
5,192,I accepted a credit card off for the Barclays...
6,75,"I would like to be credited in full, of all t..."
7,548,Can't decrease my monthly payments I am trying...
8,65,I was charged a fee both when I spoke with XX...
9,660,Having problems with customer service On XXXX ...


In [147]:
## Calculating No of Characters for test

test["char_count"]=test["GrievanceDescription"].str.len()  # also includes spaces
test[["char_count","GrievanceDescription"]].head(10)

,char_count,GrievanceDescription
0,589,I currently have a mortgage with Flag star ba...
1,456,Bank5278 Auto Finance repossessed and subsequ...
2,1974,We fell behind in our payments back in 2014 f...
3,2228,My home is on XXXX parcels of land. The first...
4,562,We mortgaged our home with XXXX on XXXX/XXXX/...
5,1189,I accepted a credit card off for the Barclays...
6,403,"I would like to be credited in full, of all t..."
7,2848,Can't decrease my monthly payments I am trying...
8,346,I was charged a fee both when I spoke with XX...
9,3899,Having problems with customer service On XXXX ...


In [148]:
# Removing Stopwords from the data 

from nltk.corpus import stopwords
stop=stopwords.words("english")
test["stopwords"]=test["GrievanceDescription"].apply(lambda x: len([x for x in x.split() if x in stop]))
test[["GrievanceDescription","stopwords"]].head()

,GrievanceDescription,stopwords
0,I currently have a mortgage with Flag star ba...,52
1,Bank5278 Auto Finance repossessed and subsequ...,28
2,We fell behind in our payments back in 2014 f...,189
3,My home is on XXXX parcels of land. The first...,196
4,We mortgaged our home with XXXX on XXXX/XXXX/...,30


In [54]:
##Number of Special Characters for test

test['hastags'] = test['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
test[['GrievanceDescription','hastags']].head(10)

,GrievanceDescription,hastags
0,I currently have a mortgage with Flag star ba...,0
1,Bank5278 Auto Finance repossessed and subsequ...,0
2,We fell behind in our payments back in 2014 f...,0
3,My home is on XXXX parcels of land. The first...,0
4,We mortgaged our home with XXXX on XXXX/XXXX/...,0
5,I accepted a credit card off for the Barclays...,0
6,"I would like to be credited in full, of all t...",0
7,Can't decrease my monthly payments I am trying...,0
8,I was charged a fee both when I spoke with XX...,0
9,Having problems with customer service On XXXX ...,0


In [149]:
## Removing Punctuation for test

test['GrievanceDescription'] = test['GrievanceDescription'].str.replace('[^\w\s]','')
test["GrievanceDescription"].head()

0     I currently have a mortgage with Flag star ba...
1     Bank5278 Auto Finance repossessed and subsequ...
2     We fell behind in our payments back in 2014 f...
3     My home is on XXXX parcels of land The first ...
4     We mortgaged our home with XXXX on XXXXXXXXXX...
Name: GrievanceDescription, dtype: object

In [150]:
## No of UpperCase words for test

test['upper'] = test['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

test[['GrievanceDescription','upper']].head(15)

,GrievanceDescription,upper
0,I currently have a mortgage with Flag star ba...,11
1,Bank5278 Auto Finance repossessed and subsequ...,5
2,We fell behind in our payments back in 2014 f...,11
3,My home is on XXXX parcels of land The first ...,56
4,We mortgaged our home with XXXX on XXXXXXXXXX...,21
5,I accepted a credit card off for the Barclays...,12
6,I would like to be credited in full of all th...,6
7,Cant decrease my monthly payments I am trying ...,28
8,I was charged a fee both when I spoke with XX...,8
9,Having problems with customer service On XXXX ...,45


In [151]:
freq = pd.Series(' '.join(test['GrievanceDescription']).split()).value_counts()[-10:] # Creating a series to check the rare words
freq

privete                 1
updatedturned           1
howrver                 1
ECROW                   1
XXXXwwwllbeancomXXXX    1
pleaeded                1
Inhumanely              1
tcalled                 1
hVe                     1
Pathfinder              1
dtype: int64

In [152]:
# Frequency of the rare words in test
freq = list(freq.index)
train['GrievanceDescription'] = train['GrievanceDescription'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['GrievanceDescription'].head()

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX I given 110000 credit limit Sears...
3    Our Mortgage sold Bank5279 XXXX 2014 They boug...
4    We started dealing Citi Mortgage back XXXXXXXX...
Name: GrievanceDescription, dtype: object

In [153]:
##Finding the total number of unique values in each column in test

test.apply(lambda x: len(x.unique()))

GrievanceID             27954
BankID                     22
State                      62
DateOfGrievance          1168
Grievance_Category         59
GrievanceDescription    24137
LineOfBusiness              4
ResolutionComments          4
Disputed                    2
DateOfResolution         1198
word_count                877
char_count               3689
stopwords                 433
upper                     258
dtype: int64

In [154]:
##Advanced Text Processing

## Generating N Grams for test

TextBlob(test['GrievanceDescription'][0]).ngrams(2)


[WordList(['I', 'currently']),
 WordList(['currently', 'have']),
 WordList(['have', 'a']),
 WordList(['a', 'mortgage']),
 WordList(['mortgage', 'with']),
 WordList(['with', 'Flag']),
 WordList(['Flag', 'star']),
 WordList(['star', 'bank']),
 WordList(['bank', 'my']),
 WordList(['my', 'XXXXXXXX']),
 WordList(['XXXXXXXX', 'payment']),
 WordList(['payment', 'was']),
 WordList(['was', 'late']),
 WordList(['late', 'reaching']),
 WordList(['reaching', 'flagstar']),
 WordList(['flagstar', 'on']),
 WordList(['on', 'XXXXXXXX15']),
 WordList(['XXXXXXXX15', 'I']),
 WordList(['I', 've']),
 WordList(['ve', 'had']),
 WordList(['had', 'this']),
 WordList(['this', 'loan']),
 WordList(['loan', 'for']),
 WordList(['for', 'over']),
 WordList(['over', 'a']),
 WordList(['a', 'year']),
 WordList(['year', 'now']),
 WordList(['now', 'and']),
 WordList(['and', 'it']),
 WordList(['it', 's']),
 WordList(['s', 'the']),
 WordList(['the', 'first']),
 WordList(['first', 'time']),
 WordList(['time', 'my']),
 WordList

In [61]:
##Term Frequency

#Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

#Therefore, we can generalize term frequency as:

#TF = (Number of times term T appears in the particular row) / (number of terms in that row)


tf2 = (test['GrievanceDescription'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf2.columns = ['words','tf']
tf2.head(5)

,words,tf
0,was,4
1,I,4
2,Bank5278,3
3,to,3
4,the,3


In [122]:
cluster_tr1

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [63]:
X_test_tfidf.shape # Checking the Shape

(53680, 53290)

In [64]:
train.head(2) 

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,hastags,upper
0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,169,944,0,31
1,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I mortgage XXXX XXXX XXXX XXXX never missed pa...,Mortgage,Closed with explanation,Yes,2016-02-10,515,2641,0,93


In [65]:
cluster_tr= pd.DataFrame({'cluster_tr':cluster_tr}) # Adding Cluster to the Dataset
s= pd.concat([cluster_tr,train],axis=1) # Adding the Column to the Train Data


In [169]:
cluster_tr1 =pd.DataFrame({'cluster_tr1':cluster_tr1})
t= pd.concat([cluster_tr1,test],axis=1)

In [160]:
t.head(2)

,cluster_tr1,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,upper
0,0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30,110.0,589.0,52.0,11.0
1,0,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03,79.0,456.0,28.0,5.0


In [125]:
tar1= pd.read_csv('Test.csv')  # Reading the Test Data

In [66]:
s.head(2)

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,hastags,upper
0,0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,169,944,0,31
1,0,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I mortgage XXXX XXXX XXXX XXXX never missed pa...,Mortgage,Closed with explanation,Yes,2016-02-10,515,2641,0,93


In [132]:
cluster_test=pd.DataFrame(cluster_test)
t= pd.concat([cluster_test,test],axis=1)
t.head(2)

,0,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30
1,0,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03


In [68]:
s=s.merge(tar,on='BankID',how='left') # Merging the BankID column from the Target For Train Data 
s.head(2)

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,hastags,upper,BankGrade
0,0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,169,944,0,31,outstanding
1,0,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I mortgage XXXX XXXX XXXX XXXX never missed pa...,Mortgage,Closed with explanation,Yes,2016-02-10,515,2641,0,93,deficient


In [170]:
m= t.merge(tar1,on='BankID',how='left') # Merging the BankID column from the Target For Test Data 
m.head(2)

,cluster_tr1,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,upper,hastags
0,0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30,110.0,589.0,52.0,11.0,0.0
1,0,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03,79.0,456.0,28.0,5.0,0.0


In [162]:
m.dtypes

cluster_tr1               int64
GrievanceID              object
BankID                   object
State                    object
DateOfGrievance          object
Grievance_Category       object
GrievanceDescription     object
LineOfBusiness           object
ResolutionComments       object
Disputed                 object
DateOfResolution         object
word_count              float64
char_count              float64
stopwords               float64
upper                   float64
dtype: object

In [86]:
s.head(2)

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,hastags,upper,BankGrade
0,2850,52943,20,23,1199,57,2,1,0,1196,172,4074,0,31,1
1,17488,52944,23,18,1199,4,2,1,1,1207,594,1830,0,93,0


In [80]:
# label Encoding for train
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [171]:
# replacing the column 'GrievanceDescription' with 'Cluster_tr1' for test data 
m["cluster_tr1"]=m["GrievanceDescription"]

In [174]:
m.head(2)

,cluster_tr1,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,upper,hastags
0,I currently have a mortgage with Flag star ba...,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",Mortgage,Closed with explanation,No,2016-01-30,110.0,589.0,52.0,11.0,0.0
1,Bank5278 Auto Finance repossessed and subsequ...,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Consumer Loan,Closed with explanation,No,2016-02-03,79.0,456.0,28.0,5.0,0.0


In [173]:
m=m.drop('GrievanceDescription',axis=1) # Dropping the Column

In [142]:
m.dtypes

0                      int64
GrievanceID           object
BankID                object
State                 object
DateOfGrievance       object
Grievance_Category    object
LineOfBusiness        object
ResolutionComments    object
Disputed              object
DateOfResolution      object
cluster_tr1           object
dtype: object

In [74]:
# dropped the column as the column is replaced by cluster_tr
s=s.drop("GrievanceDescription",axis=1)

In [141]:
m=m.drop('GrievanceDescription',axis=1)


In [175]:
m.columns

Index(['cluster_tr1', 'GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'LineOfBusiness', 'ResolutionComments',
       'Disputed', 'DateOfResolution', 'word_count', 'char_count', 'stopwords',
       'upper', 'hastags'],
      dtype='object')

In [ ]:
d.head(1)

In [176]:
m.columns

Index(['cluster_tr1', 'GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'LineOfBusiness', 'ResolutionComments',
       'Disputed', 'DateOfResolution', 'word_count', 'char_count', 'stopwords',
       'upper', 'hastags'],
      dtype='object')

In [84]:
## Transforming the data to fit the Train set 


s[['cluster_tr1']]=le.fit_transform(s[['cluster_tr']].astype('str'))
s[['GrievanceID']]=le.fit_transform(s[['GrievanceID']].astype('str'))
s[['BankID']]=le.fit_transform(s[['BankID']].astype('str'))
s[['BankGrade']]=le.fit_transform(s[['BankGrade']].astype("str"))
s[['State']]=le.fit_transform(s[['State']].astype('str'))
s[['DateOfGrievance']]=le.fit_transform(s[['DateOfGrievance']].astype('str'))
s[['Grievance_Category']]=le.fit_transform(s[['Grievance_Category']].astype('str'))
s[['LineOfBusiness']]=le.fit_transform(s[['LineOfBusiness']].astype('str'))
s[['ResolutionComments']]=le.fit_transform(s[['ResolutionComments']].astype('str'))
s[['Disputed']]=le.fit_transform(s[['Disputed']].astype('str'))
s[['DateOfResolution']]=le.fit_transform(s[['DateOfResolution']].astype('str'))
s[['word_count']]=le.fit_transform(s[['word_count']].astype('str'))
s[['char_count']]=le.fit_transform(s[['char_count']].astype('str'))
s[['hastags']]=le.fit_transform(s[['hastags']].astype('str'))
s[['BankGrade']]=le.fit_transform(s[['BankGrade']].astype('str'))

In [163]:
m.columns

Index(['cluster_tr1', 'GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'GrievanceDescription', 'LineOfBusiness',
       'ResolutionComments', 'Disputed', 'DateOfResolution', 'word_count',
       'char_count', 'stopwords', 'upper'],
      dtype='object')

In [188]:
## Transforming the data to fit the Test set

m[['cluster_tr1']]=le.fit_transform(m[['cluster_tr1']].astype('str'))
m[['GrievanceID']]=le.fit_transform(m[['GrievanceID']].astype('str'))
m[['BankID']]=le.fit_transform(m[['BankID']].astype('str'))

m[['State']]=le.fit_transform(m[['State']].astype('str'))
m[['DateOfGrievance']]=le.fit_transform(m[['DateOfGrievance']].astype('str'))
m[['Grievance_Category']]=le.fit_transform(m[['Grievance_Category']].astype('str'))
m[['LineOfBusiness']]=le.fit_transform(m[['LineOfBusiness']].astype('str'))
m[['ResolutionComments']]=le.fit_transform(m[['ResolutionComments']].astype('str'))
m[['Disputed']]=le.fit_transform(m[['Disputed']].astype('str'))
m[['DateOfResolution']]=le.fit_transform(m[['DateOfResolution']].astype('str'))
m[['word_count']]=le.fit_transform(m[['word_count']].astype('str'))
m[['char_count']]=le.fit_transform(m[['char_count']].astype('str'))
m[['hastags']]=le.fit_transform(m[['hastags']].astype('str'))
m[['upper']]=le.fit_transform(m[['upper']].astype('str'))

In [ ]:
s=pd.concat([s, tar], axis=1)

In [ ]:
s.head(1)

In [178]:
target = s[['BankGrade']]

In [88]:
target.head()

,BankGrade
0,1
1,0
2,2
3,1
4,2


In [ ]:
d= s.drop(columns=['BankGrade','BankID'])

In [ ]:
d= pd.concat([s,tar],axis=1)

In [ ]:
d.columns

In [89]:
target.head()

,BankGrade
0,1
1,0
2,2
3,1
4,2


In [180]:
# Importing train_test for train 

from sklearn.model_selection import train_test_split
target = s[['BankGrade']]
train_data_model = s[['cluster_tr1','GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','upper']]
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data_model, target, test_size=0.3)

In [189]:
# For test

from sklearn.model_selection import train_test_split
target = s[['BankGrade']]
train_data_model1 = m[['cluster_tr1','GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','upper']]
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data_model1, target, test_size=0.3)

In [190]:
train_data_model1.dtypes

cluster_tr1           int32
GrievanceID           int32
BankID                int32
State                 int32
DateOfGrievance       int32
Grievance_Category    int32
LineOfBusiness        int32
ResolutionComments    int32
Disputed              int32
DateOfResolution      int32
upper                 int32
dtype: object

### Implementing Logistic Regression

In [191]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()

from sklearn import metrics

In [92]:
# logistic regression for train data

logreg.fit(X_train, Y_train)
y_pred=logreg.predict(X_valid)
accuracy_lr = metrics.accuracy_score(Y_valid, y_pred)

In [93]:
accuracy_lr

0.6503974167908594

In [ ]:
train_model

In [192]:
# Logistic Regression for test
logreg.fit(X_train, Y_train)
y_pred1=logreg.predict(X_valid)
accuracy_lr1 = metrics.accuracy_score(Y_valid, y_pred)

In [193]:
accuracy_lr1

0.5051539990064581

### The Generated for the Test Data comes at 50.58%  using Logistic Regression

In [ ]:
# Creating the DataFrame For Train & Test
train_data_model = d[['cluster_tr','GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','upper']]
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data_model, target, test_size=0.3)

### Implementing Random Forest

In [95]:
# Random Forest Classifier for train
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data_model, target, test_size=0.3)
from sklearn.ensemble import RandomForestClassifier


In [194]:
# Random Forest Classifier for test
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data_model1, target, test_size=0.3)
from sklearn.ensemble import RandomForestClassifier

In [96]:
clf=RandomForestClassifier(n_estimators=100) # Fitting the Classifier
clf.fit(X_train,Y_train)
y_pred=clf.predict(X_valid)


In [97]:
from sklearn import metrics
print("Accuracy for train:",metrics.accuracy_score(Y_valid, y_pred))

Accuracy: 0.9769622454048683


In [195]:
# Predicting on Test Data
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,Y_train)
y_pred1=clf.predict(X_valid)

In [196]:
from sklearn import metrics
print("Accuracy for test:",metrics.accuracy_score(Y_valid, y_pred))

Accuracy for test: 0.5058991554893194


### The Accuracy Generated for the Test Data comes at 50.58%  using Random Forest Method

In [204]:
test_df = pd.DataFrame({'Output':y_pred1})
test_df.to_csv('m', sep='\t', encoding='utf-8')